# 1 IMPORTS

In [2]:
import pandas as pd
import inflection

## 1.1 Definitions 

## 1.2 Loading Data

In [3]:
# coletando os dados e salvando localmente
url = 'https://raw.githubusercontent.com/sthemonica/alura-voz/main/Dados/Telco-Customer-Churn.json'
df_raw = pd.read_json(url)
df_raw.to_json('../data/customers.json')


# 2 DATA DESCRIPTION

## 2.1 Data Preview

Após a importação dos dados, verificamos que há informações aninhadas em formato json. Portanto, para acessar esses dados e termos o nosso dataframe definitivo, utilizaremos ferramentas que serão mostradas nas próximas sessões.

In [4]:
df_raw.head(3)

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


## 2.2 Data Format and Types

In [5]:
# coletando informacoes dos jsons
columns = ['customer', 'phone', 'internet', 'account']
for column in columns:
    df_temp = pd.json_normalize(data=df_raw[column], sep='_')
    df_raw = pd.concat([df_raw, df_temp], axis=1)

# removendo colunas duplicadas
df_raw = df_raw.loc[:,~df_raw.columns.duplicated()]

# padronizando os headers para snake case
cols_snake = list( map( lambda x: inflection.underscore( x ), df_raw.columns ) )
df_raw.columns = cols_snake

# removendo as colunas com os formatos json
df_raw.drop(columns=['customer', 'phone', 'internet', 'account'],inplace=True)

# transformando valores vazios e ausentes em NA
df_raw = df_raw.mask(df_raw == ' ').mask(df_raw == '')

In [6]:
# removendo 11 linhas que estão com valores vazios em charges_total para converter em float
df_raw.dropna(subset=['charges_total'], inplace=True)
df_raw['charges_total'] = df_raw['charges_total'].astype('float')

In [7]:
df_raw.head(3)

,customer_id,churn,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,charges_monthly,charges_total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85


In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7256 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7256 non-null   object 
 1   churn              7032 non-null   object 
 2   gender             7256 non-null   object 
 3   senior_citizen     7256 non-null   int64  
 4   partner            7256 non-null   object 
 5   dependents         7256 non-null   object 
 6   tenure             7256 non-null   int64  
 7   phone_service      7256 non-null   object 
 8   multiple_lines     7256 non-null   object 
 9   internet_service   7256 non-null   object 
 10  online_security    7256 non-null   object 
 11  online_backup      7256 non-null   object 
 12  device_protection  7256 non-null   object 
 13  tech_support       7256 non-null   object 
 14  streaming_tv       7256 non-null   object 
 15  streaming_movies   7256 non-null   object 
 16  contract           7256 

## 2.3 NA Checking and Treatment

Agora que conseguimos ter acesso às informações dos jsons aninhados, faremos o tratamento de valores nulos/faltantes.

In [9]:
df_raw.isna().sum()

customer_id            0
churn                224
gender                 0
senior_citizen         0
partner                0
dependents             0
tenure                 0
phone_service          0
multiple_lines         0
internet_service       0
online_security        0
online_backup          0
device_protection      0
tech_support           0
streaming_tv           0
streaming_movies       0
contract               0
paperless_billing      0
payment_method         0
charges_monthly        0
charges_total          0
dtype: int64

In [10]:
# deletando as linhas com valores nulos em churn. Como é nossa variável resposta, não
# temos como inferir valores.
df_raw.dropna(inplace=True)

In [11]:
df_raw.isna().sum()

customer_id          0
churn                0
gender               0
senior_citizen       0
partner              0
dependents           0
tenure               0
phone_service        0
multiple_lines       0
internet_service     0
online_security      0
online_backup        0
device_protection    0
tech_support         0
streaming_tv         0
streaming_movies     0
contract             0
paperless_billing    0
payment_method       0
charges_monthly      0
charges_total        0
dtype: int64

## 2.4 Discriptive Statistics

Nesta seção faremos uma rápida análise estatística para identificar se temos valores que precisam de uma maior atenção (valores negativos, outliers, erros de digitação, etc).

### Numerical Attributes

In [12]:
num_att = df_raw.select_dtypes(['int64', 'float64'])
# getting skew
skew = pd.DataFrame(num_att.apply(lambda x: x.skew())).round(2).T
skew.rename(index={0: 'skew'}, inplace=True)

# getting kurtosis
kurtosis = pd.DataFrame(num_att.apply(lambda x: x.kurtosis())).round(2).T
kurtosis.rename(index={0: 'kurtosis'}, inplace=True)

# numerical features 
describe = df_raw.describe().round(2)
m = pd.concat([describe, skew, kurtosis], axis=0)
m

,senior_citizen,tenure,charges_monthly,charges_total
count,7032.00,7032.00,7032.00,7032.00
mean,0.16,32.42,64.80,2283.30
std,0.37,24.55,30.09,2266.77
min,0.00,1.00,18.25,18.80
25%,0.00,9.00,35.59,401.45
50%,0.00,29.00,70.35,1397.48
75%,0.00,55.00,89.86,3794.74
max,1.00,72.00,118.75,8684.80
skew,1.83,0.24,-0.22,0.96
kurtosis,1.35,-1.39,-1.26,-0.23


### Categorical Attributes

In [13]:
cat_att = df_raw.select_dtypes(exclude=['int64', 'float64'])
cat_att.apply( lambda x: x.unique().shape[0] )

customer_id          7032
churn                   2
gender                  2
partner                 2
dependents              2
phone_service           2
multiple_lines          3
internet_service        3
online_security         3
online_backup           3
device_protection       3
tech_support            3
streaming_tv            3
streaming_movies        3
contract                3
paperless_billing       2
payment_method          4
dtype: int64

In [16]:
for column in df_raw.columns:
    print(df_raw[column].unique())

['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
['No' 'Yes']
['Female' 'Male']
[0 1]
['Yes' 'No']
['Yes' 'No']
[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 37 49 66 67
 20 43 59 12 27  2 25 29 14 35 64 39 40 11  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 31 36 17 18 51 38 42]
['Yes' 'No']
['No' 'Yes' 'No phone service']
['DSL' 'Fiber optic' 'No']
['No' 'Yes' 'No internet service']
['Yes' 'No' 'No internet service']
['No' 'Yes' 'No internet service']
['Yes' 'No' 'No internet service']
['Yes' 'No' 'No internet service']
['No' 'Yes' 'No internet service']
['One year' 'Month-to-month' 'Two year']
['Yes' 'No']
['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
[65.6  59.9  73.9  ... 91.75 68.8  67.85]
[ 593.3   542.4   280.85 ...  742.9  4627.65 3707.6 ]


Para as features numéricas, não identificamos valores discrepantes ou possivelmente errados. Porém, para as features categóricas, observamos que algumas features apresentam valores redundantes ('No' e 'No xxxx service'). Para esse primeiro ciclo CRISP-DM/DS, iremos considerar que esses dois valores transmitem a mesma informação e iremos padronizá-los na seção 5.

# 3 DATAFRAME FILTERING

## 3.1 Line Filtering

## 3.2 Column Selection

# 4 QUESTIONS & HYPOTHESIS

# 5 FEATURE ENGINEERING

In [75]:
# ------- CONFERIR A EFETIVIDADE DESSA MODIFICAÇÃO NOS PRÓXIMOS CICLOS --------
# transformando os valores "no xxxx service em no"
for column in df_raw.columns[8:16]:
    df_raw.loc[:,column] = df_raw[column].apply(lambda x: 'No' if 'No' in x else x)
# -----------------------------------------------------------------------------

# criando feature de gasto diário partindo do charges_total
df_raw.insert(loc=18, column='daily_charge', value=df_raw['charges_total']/(df_raw['tenure']*30))

# 6 EXPLORATORY DATA ANALYSIS

## 6.1 Univariate Analysis

### 6.1.1 Response Variable

### 6.1.2 Numerical Variable

### 6.1.3 Categorical Variable

## 6.2 Bivariate Analysis

## 6.3 Multivariate Analysis


### 6.3.1 Numerical Attributes

### 6.3.2 Categorical Attributes

# 7 DATA PREPARATION

## 7.2 Normalization

## 7.3 Rescaling

## 7.4 Transformation

### 7.4.1 Encoding

# 8 FEATURE SELECTION

## 8.1 Split dataframe into training and validation dataset

## 8.3 Selected Features

# 9 MACHINE LEARNING MODELLING

# 10 CROSS-VALIDATION

# 11 HYPERPARAMETER FINE TUNNING

## 11.1 Random Search

## 11.2 Final Model

# 12 BUSINESS PERFORMANCE

# 13 NEXT STEPS